In [31]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain_core.documents import Document
import concurrent.futures
from langchain_aws import BedrockEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from pathlib import Path
current_dir = Path.cwd()
root_path = current_dir.parent.parent  # 상위 디렉토리
sys.path.insert(0, str(root_path))
sys.path.insert(0, str(root_path / "shared"))
from opensearch_con import OpenSearchEmbeddingProcessor 
from langchain_aws import ChatBedrock


In [2]:
#create opensearch connection and set Bedrock Embedding
opensearh= OpenSearchEmbeddingProcessor()

In [3]:
#embedding test
opensearh.get_embedding("안녕하세요?")[:5]

[-0.08115597069263458,
 0.0458092987537384,
 -0.03429197520017624,
 0.00505836820229888,
 0.04746834188699722]

In [4]:
#get data path
data_path=str(opensearh.get_data_path())
print(data_path)

/home/sagemaker-user/smus_agentic_core/data/raw/basic_aws_dictionary.json


In [5]:
#get json data
data = opensearh.read_json(data_path)["char_chunks"]

In [6]:
#check chunk shape and features
opensearh.check_data_property(data)

data ->  {'chunk_id': 0, 'content': 'Amazon Web Services 개요 AWS 백서 Amazon Web Services 개요 게시일: 2024년 8월 27 일(문서 이력 ) Amazon Web Services는 컴퓨팅, 스토리지, 데이터베이스, 분석, 네트워킹, 모바일, 개발자 도구, 관리 도구, IoT, 보안 및 엔터프라이즈 애플리케이션을 포함한 다양한 글로벌 클라우드 기반 제품 세트를 제 공합니다. 온디맨드, 몇 초 만에 사용 가능, pay-as-you-go 데이터 웨어하우징에서 배포 도구, 디렉터 리, 콘텐츠 전송에 이르기까지 200개 이상의 AWS 서비스를 사용할 수 있습니다. 선결제 고정 비용 없이 새 서비스를 빠르게 프로비저닝할 수 있습니다. 이를 통해 기업, 스타트업, 중소 기업 및 공공 부문의 고객은 변화하는 비즈니스 요구 사항에 신속하게 대응하는 데 필요한 구성 요소에 액세스할 수 있습니다. 이 백서에서는의 이점에 대한 개요를 AWS 클라우드 제공하고 플랫폼을 구성하 는 서비스를 소개합니다. 소개 2006년에 Amazon Web Services(AWS)는 기업에 웹 서비스로 IT 인프라 서비스를 제공하기 시작했습 니다. 이제는 클라우드 컴퓨팅이라고 합니다. 클라우드 컴퓨팅의 주요 이점 중 하나는 선결제 자본 인 프라 비용을 비즈니스에 따라 조정되는 낮은 가변 비용으로 바꿀 수 있는 기회입니다. 클라우드를 사 용하면 기업은 더 이상 서버 및 기타 IT 인프라를 몇 주 또는 몇 달 전에 계획하고 조달할 필요가 없습니 다. 대신 몇 분 만에 수백 또는 수천 개의 서버를 즉시 가동하고 결과를 더 빠르게 제공할 수 있습니다. 오늘날는 전 세계 190개 국가에서 수십만 개의 비즈니스를 지원하는 클라우드에서 매우 안정적이고 확 장 가능하며 저렴한 인프라 플랫폼을 AWS 제공합니다. 이 동영상에서는 수백만 명의 고객이 클라우드 컴퓨팅의 효율성을 활용하는 AWS 방법을 살펴봅니 다.란 무엇인가요 AWS? | Amazon We

In [7]:
#change content into vector via bedrock embedding
vector=opensearh.get_embedding_with_key(data[0], "content")
print(vector[:5])

[-0.03377218544483185, -0.02527015656232834, 0.014535900205373764, -0.08748951554298401, 0.015982862561941147]


In [8]:

# Create index
index_name = "aws-document-chunks"
    

# Index mapping
index_mapping = {
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "nori_analyzer"
            },
            "content_embedding": {
                "type": "knn_vector",
                "dimension": 1024
            },
            "summary": {
                "type": "text",
                "analyzer": "nori_analyzer"
            },
            "expected_question": {
                "type": "text",
                "analyzer": "nori_analyzer"
            },
            "keywords": {
                "type": "keyword"
            }
        }
    },
    "settings": {
        "index": {
            "knn": True,
            "knn.algo_param.ef_search": 100,
            "number_of_shards": 3,
            "number_of_replicas": 2
        },
        "analysis": {
            "analyzer": {
                "nori_analyzer": {
                    "tokenizer": "nori_tokenizer",
                    "filter": ["nori_stop", "lowercase"]
                }
            },
            "filter": {
                "nori_stop": {
                    "type": "nori_part_of_speech",
                    "stoptags": ["J", "JKS", "JKB", "JKO", "JKG", "JKC", "JKV", "JKQ", "JX", "JC"]
                }
            }
        }
    }
}




In [10]:
#create index
opensearh.create_index(index_name,index_mapping )

#delete index
# opensearh.delete_index(index_name)

Index aws-document-chunks created successfully


In [11]:
#generate text and embedding feature
def preprocess_content(process, data):
    feature=data
    feature_embedding=process.get_embedding(data)
    return feature, feature_embedding

In [12]:
for idx, chunk in enumerate(data):
    #content 
    content, content_embedding=preprocess_content(opensearh, chunk["content"])
    
    #summary
    summary, summary_embedding=preprocess_content(opensearh, chunk["summary"])

    #expected_question
    expected_question=",".join(chunk["expected_questions"])
    expected_question, expected_question_embedding=preprocess_content(opensearh, expected_question)

    #keyword
    keywords = chunk["keywords"]

    #mapping features into opensearch index format
    document = {
            "content": content,
            "content_embedding": content_embedding,
            "summary" : summary, 
            "expected_question" : expected_question,
            "keywords" : keywords 
        }

    opensearh.save_data(idx, document)


Document 0 indexed successfully
Document 50 indexed successfully
Document 100 indexed successfully
Document 150 indexed successfully
Document 200 indexed successfully


In [ ]:
#vector 유사도 검색 
def vector_search(client, query, k=3):
    try:
        # KNN vector search query

        query_vector= client.get_embedding(query)
        vector_search = {
            "size": k,
            "_source": {
                "excludes": ["content_embedding"]  # Exclude vector field from results
            },
            "query": {
                "knn": {
                    "content_embedding": {
                        "vector": query_vector,
                        "k": k
                    }
                }
            }
        }

        # Execute the search
        response = client.os_client.search(
            index=client.index_name,
            body=vector_search
        )

        documents = []
        for res in response["hits"]["hits"]:
            source = res['_source']
            page_content = {k: source[k] for k in source if k != "vector"}
            metadata = {"id": res['_id']}  # Add metadata with a unique identifier
            score = res['_score']  # Get the match score from the search result
            documents.append((Document(page_content=json.dumps(page_content, ensure_ascii=False), metadata=metadata), score))
        return documents
            
    except Exception as e:
        print(f"Search error: {str(e)}")
        return None

In [29]:
#결과 검색
result =vector_search(opensearh, "Amazon Appflow에 대해 설명해줘", k = 5)

In [30]:
result

[(Document(metadata={'id': 'aws_doc_26'}, page_content='{"summary": "Amazon AppFlow는 SaaS 애플리케이션과 AWS 서비스 간 데이터를 안전하게 전송하는 완전관리형 통합 서비스이며, AWS B2B Data Interchange는 EDI 문서를 JSON/XML 형식으로 자동 변환하여 데이터 통합을 간소화하는 서비스입니다.", "keywords": ["Amazon AppFlow", "AWS B2B Data Interchange", "SaaS 애플리케이션", "EDI", "데이터 전송", "완전관리형 통합 서비스", "AWS PrivateLink", "데이터 변환", "JSON", "XML"], "expected_question": "Amazon AppFlow에서 지원하는 SaaS 애플리케이션에는 어떤 것들이 있나요?,Amazon AppFlow의 데이터 흐름 실행 빈도는 어떻게 설정할 수 있나요?,AWS B2B Data Interchange에서 지원하는 EDI 표준 형식은 무엇인가요?,Amazon AppFlow에서 AWS PrivateLink를 통한 보안은 어떻게 구현되나요?,AWS B2B Data Interchange의 로우 코드 인터페이스는 어떤 기능을 제공하나요?", "content": "못된 경우 빠르게 문제를 진단하고 디버깅할 수 있습니다. 코드를 작성하지 않고도 단계를 변경하고 추가할 수 있으므로 애플리 케이션을 쉽게 발전시키고 더 빠르게 혁신할 수 있습니다. Amazon AppFlow Amazon AppFlow 는 Salesforce, Zendesk, Slack, ServiceNow와 같은 Software-as-a-Service(SaaS) 애 플리케이션과 Amazon S3, Amazon Redshift와 같은 AWS 서비스 간에 몇 번의 클릭만으로 데이터를 안전하게 전송할 수 있는 완전관리형 통합 서비스입니다. Amazon AppFlow를 사용하면 일정에 따라, 비즈니스 이벤트에 대응하여

In [32]:
class RagLLM:
    """RagLLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None

In [33]:
rag = RagLLM()

In [34]:
#template 생성
template_lambda = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:
{document}

Conversation:
Human: {question}
AI:"""

# Create the prompt template
prompt_lambda = PromptTemplate(
    input_variables=["document", "question"], 
    template=template_lambda
)

In [37]:
#how to use opensearch for langchain
chain_lambda_rag = (
    {
        "document": lambda x: vector_search(opensearh, x["question"]),
        "question": itemgetter("question")
    }
    | prompt_lambda
    | rag.llm
    | StrOutputParser()
)


In [39]:
async for chunk in chain_lambda_rag.astream({"question": "Amazon S3와 EMR을 연동하는 방법에 대해 설명해줘"}):
    print(chunk, end="")

안녕하세요! 제공된 정보를 바탕으로 Amazon S3와 EMR에 대해 설명드릴 수 있지만, 아쉽게도 두 서비스를 직접 연동하는 구체적인 방법에 대한 정보는 제공된 문서에 포함되어 있지 않습니다.

하지만 각 서비스의 특징을 통해 연동 가능성에 대해 말씀드릴 수 있습니다:

**Amazon S3의 특징:**
- 업계 최고의 확장성, 데이터 가용성, 보안 및 성능을 제공하는 객체 스토리지 서비스
- 99.999999999%(11 9s)의 내구성을 제공
- 웹사이트, 모바일 애플리케이션, 백업 및 복원, 아카이브, 빅 데이터 분석 등 다양한 사용 사례 지원
- 다양한 액세스 패턴에 맞는 여러 스토리지 클래스 제공 (S3 Intelligent-Tiering, S3 표준, S3 Express One Zone 등)

**Amazon EMR의 특징:**
- Apache Spark, Apache Hive, Apache HBase, Apache Flink 등 오픈소스 도구를 사용한 클라우드 빅데이터 플랫폼
- 페타바이트 규모의 분석을 기존 온프레미스 솔루션 비용의 절반 미만으로 실행 가능
- 표준 Apache Spark보다 3배 이상 빠른 성능

두 서비스 모두 빅데이터 분석을 지원한다는 공통점이 있어 연동 가능성이 높아 보이지만, 구체적인 연동 방법, 설정 절차, API 사용법 등에 대한 상세한 정보는 제공된 문서에서 찾을 수 없습니다.

더 구체적인 연동 방법에 대해서는 AWS 공식 문서나 기술 가이드를 참조하시는 것을 권장드립니다.

In [40]:
#RunnableLambda안에서 실행할 함수
def get_document_runnable(data):
    print(data)
    # Make sure this function returns a string
    return vector_search(opensearh, data)


In [42]:
#input을 넣을때 함수 실행 (RunnableLambda 실행)
chain_runnable_rag = (
    {
        "document": itemgetter("question") | RunnableLambda(get_document_runnable),
        "question": itemgetter("question")
    }
    | prompt_lambda
    | rag.llm
    | StrOutputParser()
)


In [48]:
async for chunk in chain_runnable_rag.astream({"question": "Amazon S3 glacier에 대해 설명해줘"}):
    print(chunk, end="")

Amazon S3 glacier에 대해 설명해줘
Amazon S3 Glacier에 대해 설명드리겠습니다.

Amazon S3에서는 아카이브 데이터를 위한 여러 Glacier 스토리지 클래스를 제공합니다:

## S3 Glacier 스토리지 클래스 종류

**1. S3 Glacier Instant Retrieval**
- 즉각적인 액세스가 필요한 아카이브 데이터를 위한 스토리지 클래스입니다
- 아카이브 용도이지만 빠른 검색이 필요한 경우에 적합합니다

**2. S3 Glacier Flexible Retrieval (이전 Amazon Glacier)**
- 즉각적인 액세스가 필요하지 않은 드물게 액세스하는 장기 데이터를 위한 스토리지 클래스입니다
- 자주 접근하지 않는 장기 보관 데이터에 적합합니다

**3. Amazon Glacier Deep Archive**
- 클라우드에서 가장 저렴한 스토리지로 제공됩니다
- 장기 아카이브 및 디지털 보존을 위한 용도입니다
- 데이터 검색에 몇 시간이 소요됩니다

## 주요 특징

이러한 Glacier 스토리지 클래스들은 Amazon S3의 99.999999999%(11 9s) 내구성을 제공하며, 백업 및 복원, 아카이브 등의 사용 사례에 적합합니다. 또한 S3 수명 주기 정책을 설정하면 애플리케이션 변경 없이 데이터가 자동으로 이러한 아카이브 스토리지 클래스로 전송될 수 있습니다.

In [49]:
#nested llm chain
def get_document_runnable_nested(data):
    print(data)
    # Make sure this function returns a string
    return { 
             "question" : data,
             "document": vector_search(opensearh, data)
            }


In [50]:
template_rerank = """You are an AI document processor that organizes and summarizes information.

TASK:
1. Analyze the user's question to understand what information is relevant
2. Review the provided document and identify sections that relate to the question
3. Extract ONLY the parts of the document that are relevant to the question
4. Organize these relevant parts into a clear, concise summary
5. Remove any redundant or irrelevant information
6. Format the output as a structured summary of key points
7. DO NOT answer the question or provide any additional information not in the document
8. DO NOT include your own analysis or opinions

Document to Process:
{document}

Question for Context (use only to determine relevance):
{question}

OUTPUT INSTRUCTIONS:
- Return ONLY the organized and summarized relevant information
- Do not include any introduction, explanation, or conclusion
- Do not address the question directly
- Format as bullet points or short paragraphs of key information

Organized Relevant Information:"""




# Create the prompt template
prompt_rerank = PromptTemplate(
    input_variables=["document", "question"], 
    template=template_rerank
)

In [51]:
rerank_chain = (
    {
        "document": (
                    itemgetter("question") 
                    | RunnableLambda(get_document_runnable_nested) 
                    | prompt_rerank
                    | rag.llm
                    | StrOutputParser()
                    ),
        "question": itemgetter("question")
    }
    | prompt_lambda
    | rag.llm
    | StrOutputParser()
)


In [52]:
async for chunk in rerank_chain.astream({"question": "Amazon S3 glacier에 대해 설명해줘"}):
    print(chunk, end="")

Amazon S3 glacier에 대해 설명해줘
Amazon S3 Glacier에 대해 자세히 설명해드리겠습니다!

Amazon S3 Glacier는 아카이브 데이터를 위한 스토리지 서비스로, 현재 세 가지 주요 스토리지 클래스로 구성되어 있습니다:

## 주요 스토리지 클래스

**1. S3 Glacier Instant Retrieval**
- 즉각적인 액세스가 필요한 아카이브 데이터를 위한 스토리지 클래스입니다
- 아카이브 데이터임에도 불구하고 바로 접근할 수 있는 것이 특징입니다

**2. S3 Glacier Flexible Retrieval** (이전 Amazon Glacier)
- 즉각적인 액세스가 필요하지 않은 드물게 액세스하는 장기 데이터를 위한 스토리지 클래스입니다
- 기존의 Amazon Glacier 서비스가 이 이름으로 변경되었습니다

**3. Amazon Glacier Deep Archive**
- 클라우드에서 가장 저렴한 스토리지입니다
- 장기 아카이브 및 디지털 보존을 위한 용도로 사용됩니다
- 몇 시간 만에 검색이 가능합니다

## 주요 특징

**자동화된 관리**: S3 수명 주기 정책과 연동하여 애플리케이션 변경 없이 데이터가 자동으로 다른 스토리지 클래스로 전송됩니다. 이를 통해 수명 주기 동안 데이터를 자동으로 관리할 수 있습니다.

**비용 효율성**: 다양한 액세스 패턴에 가장 저렴한 스토리지를 제공하도록 특별히 설계되었으며, 아카이브 스토리지를 포함한 거의 모든 사용 사례에 적합합니다.

**유연한 선택**: 워크로드의 데이터 액세스, 복원력 및 비용 요구 사항에 따라 적절한 스토리지 클래스를 선택할 수 있습니다.